In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
import numpy as np
def log_no(q, ps, ns):
    return -np.sum(ns * np.log(1 - q * ps))
for i in range(5000):
    N = 2#np.random.randint(low=100, high=2000)
    ps = np.random.uniform(size=N)
    n_a = np.random.randint(low=0, high=1000, size=N)
    n_b = np.random.randint(low=0, high=1000, size=N)
    q0 = np.random.uniform(low=0, high=1e-3,size=1)[0]
    no_a0 = log_no(q0, ps, n_a)
    no_b0 = log_no(q0, ps, n_b)
    order0 = (no_a0 > no_b0)
    if order0:
        err_msg = f'{i}: q0 = {q0}, no_a0 = {no_a0} > {no_b0} = no_b0\n'
    else:
        err_msg = f'{i}: q0 = {q0}, no_b0 = {no_b0} > {no_a0} = no_a0\n'

    for q in np.random.uniform(low=0, high=q0,size=1000):
        no_a = log_no(q, ps, n_a)
        no_b = log_no(q, ps, n_b)
        order = (no_a > no_b)       
        if order0 != order:
            if order:
                err_msg = err_msg + f'q = {q}, no_a = {no_a} > {no_b} = no_b\n'
            else:
                err_msg = err_msg + f'q = {q}, no_b = {no_b} > {no_a} = no_a\n'
            print(err_msg)
            
            assert False
            
            



In [51]:
ps

array([0.70277063, 0.96708173, 0.10522932, 0.32292721, 0.40809309,
       0.26800538, 0.26328423, 0.17746625, 0.39161197, 0.54271717,
       0.02029058, 0.03829489, 0.80053014, 0.46572665, 0.16372734,
       0.39007532, 0.138969  , 0.57412519, 0.93601435, 0.78321403,
       0.82547225, 0.51892545, 0.8647809 , 0.41290802, 0.9688102 ,
       0.42703265, 0.18126513, 0.88633673, 0.91985491, 0.70345134,
       0.09803319, 0.34420212, 0.81162517, 0.49018873, 0.12979581,
       0.55503191, 0.45456399, 0.47806152, 0.51646993, 0.01996253,
       0.9291863 , 0.88471195, 0.55243128, 0.3398955 , 0.02552667,
       0.82338752, 0.58726727, 0.25747501, 0.40564361, 0.91640258,
       0.54756142, 0.43301652, 0.29939555, 0.24280066, 0.74647451,
       0.00221339, 0.69540233, 0.37523755, 0.6155519 , 0.47294686,
       0.58865588, 0.27012834, 0.13521662, 0.19830773, 0.2955071 ,
       0.95845364, 0.93239248, 0.79104165, 0.03226918, 0.49085817,
       0.17585919, 0.41421442, 0.71335066, 0.26448876, 0.59924

In [27]:
import math
import numpy as np
def get_p2(p1, n1, n2, q=1e-6):
    enum = np.power( (1 - q*(1-p1)), n2/n1) - 1
    return 1 + enum / q

def f(p, n, q):
    return (1 - q * (1-p))**n

p1 = 0.01
n1 = 10000
n2 = 20009
q = 1e-6
p2 = get_p2(p1=p1, n1=n1, n2=n2, q=q)

for q in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8]:
    print(f(p1, n1, q) - f(p2, n2, q))

0.3525706212972674
0.23297794801271188
0.029009289218990708
0.0029662085213627565
0.00029728264841577534
